In [1]:
import geopandas as gpd
import pandas as pd
import os

In [2]:
path = r"T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles"

In [3]:
file = '2020_emme_links.shp'

In [4]:
df = pd.read_excel(os.path.join(path, 'CLMPO Lane Miles.xlsx'), skiprows=1)
df.rename(columns={'Unnamed: 0':'Functional Classification'}, inplace=True)

In [5]:
df_mapping = pd.DataFrame({'fed_class': df['Functional Classification'].values})
sort_mapping = df_mapping.reset_index().set_index('fed_class')

In [6]:
def aggLaneMiles(file):
    gdf = gpd.read_file(os.path.join(path,'output',file))
    aggdata = gdf[['miles', 'fed_class']].groupby('fed_class').agg('sum')
    aggdata = {'fed_class':aggdata.index.values,'miles':aggdata.miles.values}
    aggdata = pd.DataFrame(aggdata)
    aggdata['fed_class'] = aggdata['fed_class'].str.replace('and','&')
    aggdata['order'] = aggdata['fed_class'].map(sort_mapping['index'])
    return aggdata.dropna().sort_values('order').miles.values

In [7]:
aggLaneMiles(file)

array([  87.45753493,   94.43993019,  120.30960597,  347.03139523,
        253.3708006 ,  185.31155025, 2079.56250318])

In [8]:
df['Base Year'] = aggLaneMiles('2020_emme_links.shp')
df['Future Year'] = aggLaneMiles('2045_emme_links.shp')

In [9]:
df

,Functional Classification,Base Year,Future Year
0,Interstate,87.457535,89.559305
1,Other Freeways & Expressways,94.439930,104.181209
2,Other Principal Arterial,120.309606,126.266413
3,Minor Arterial,347.031395,371.100859
4,Major Collector,253.370801,283.479892
5,Minor Collector,185.311550,192.182215
6,Local,2079.562503,2179.535656


In [10]:
df.to_csv(os.path.join(path, 'output', 'CLMPO_Lane_Miles.csv'), index=False)